In [2]:
import numpy as np

In [3]:
def init(self):
    self.alpha = 0.08 # 8 процента
    self.holdPeriod = 350 # не менее 20
    self._tickSize = 'm5'
    self.currIndex = 0

    
    self.up = []
    self.down = []
    self.rsi = 50
    self.rsiIndex = 0
    self.rsiAlpha = 2 / (3 * 12 + 1)
    self.rsiUpEma = 0
    self.rsiDownEma = 0
    
    self.sma = 0
    self.smaArray = []
    
    self.stohastic_K_fast = []
    self.stohastic_K_full = []
    self.stohastic_D_full = []
    self.stohasticIndicator = 0
    
def tick(self, data):
    self.currIndex += 1
    #-----------------------------rsi-----------------------------------
    if self.currIndex > 36:
        self.up = []
        self.down = []
        for i in range(-37, -1):
            diff = data['close'][i + 1] - data['close'][i] 
            if diff > 0:
                self.up.append(diff)
                self.down.append(0)
            else:
                self.up.append(0)
                self.down.append(-diff)
        self.rsiUpEma = self.up[0]
        for i in range(1, 36):
            self.rsiUpEma = self.rsiAlpha * self.up[i] + (1 - self.rsiAlpha) * self.rsiUpEma
        self.rsiDownEma = self.down[0]
        for i in range(1, 36):
            self.rsiDownEma = self.rsiAlpha * self.down[i] + (1 - self.rsiAlpha) * self.rsiDownEma
        self.rsi = 100*(self.rsiUpEma/(self.rsiUpEma + self.rsiDownEma))
        if self.rsi > 80:
            self.rsiIndex = -1
        elif self.rsi < 20:
            self.rsiIndex = 1
        else:
            self.rsiIndex = 0
    #-----------------------------rsi-----------------------------------
    
    #-----------------------------sma-----------------------------------
    if self.currIndex > 1561:
        self.sma = np.average(data['close'][-1561:])
    #-----------------------------sma-----------------------------------
    
    #--------------------------stohastic(f_p, s_p, t_p)--------------------------------
    f_p = 2 * 12
    s_p = 3 * 12
    t_p = 3 * 12
    
    self.stohastic_K_fast.append(100 * (data['close'][-1] - np.min(data['low'][-min(f_p, len(data['low'])):])) / (np.max(data['high'][-min(f_p, len(data['high'])):]) - np.min(data['low'][-min(f_p, len(data['low'])):])))
    self.stohastic_K_full.append(np.average(self.stohastic_K_fast[-min(s_p, len(self.stohastic_K_fast)):]))
    self.stohastic_D_full.append(np.average(self.stohastic_K_full[-min(t_p, len(self.stohastic_K_full)):]))
    
    if self.currIndex > f_p + s_p + t_p:
        if ((self.stohastic_K_full[-2] > self.stohastic_D_full[-2]) ^ (self.stohastic_K_full[-1] < self.stohastic_D_full[-1])) and (self.stohastic_K_full[-2] < 30 or self.stohastic_K_full[-1] < 30) and (self.stohastic_D_full[-2] < 30 or self.stohastic_D_full[-1] < 30):
            self.stohasticIndicator = 1
        elif ((self.stohastic_K_full[-2] > self.stohastic_D_full[-2]) ^ (self.stohastic_K_full[-1] < self.stohastic_D_full[-1])) and (self.stohastic_K_full[-2] > 70 or self.stohastic_K_full[-1] > 70) and (self.stohastic_D_full[-2] > 70 or self.stohastic_D_full[-1] > 70):
            self.stohasticIndicator = -1
        else:
            self.stohasticIndicator = 0
    #--------------------------stohastic--------------------------------

    if ((self.rsiIndex == 1) and (data['close'][-1] > self.sma) and (self.stohasticIndicator == 1)):
    	order('buy', takeProfit=self.alpha/2, stopLoss=self.alpha/4, holdPeriod=self.holdPeriod)
    elif self.rsiIndex == -1 and data['close'][-1] < self.sma and self.stohasticIndicator == 1:
    	order('sell', takeProfit=self.alpha, stopLoss=self.alpha/4, holdPeriod=self.holdPeriod)  
        
        
        
        
import bcsquants
from bcsquants import order


result = bcsquants.getBacktestResult(init, tick)
bcsquants.showBacktestResult(result)

SBERP, SBER
Не забудьте вы можете посмотреть тики и заявки в соответствующих файлах
tickFile = data/order/TICKER_m5_tick.csv
orderFile = data/order/TICKER_m5_order.csv


,scale,sumProcent,maxDrawdown,numDeals,sumTakeProfit,sumHoldPeriod,sumStopLoss
SBERP,-17.37,-10.91,10.32,14,12.92,-1.19,-22.64
SBER,-20.21,-18.01,16.02,13,0.00,2.35,-20.37


In [4]:
% pylab
bcsquants.plotChart(result, 'BRX')

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
